In [1]:
from dotenv import load_dotenv
load_dotenv()

True

In [5]:
from clearml import PipelineController
pipe = PipelineController(
  name="ETL", project="my project"
)

ClearML pipeline page: http://localhost:8080/pipelines/7f51a955686e46aeaf35b96c96a4e9e8/experiments/e64da58832624df3895b30fb5d98031e


In [ ]:
class UserDoc:
    username : str
    id : int
    def __init__(self,name:str):
        username=name

In [ ]:
myclient = pymongo.MongoClient("mongodb://localhost:27017/")
local = myclient["local"]
coll = local["ETL"]

In [21]:
from bs4 import BeautifulSoup
import requests
url = "https://www.ros.org/blog/getting-started/"
response = requests.get(url)
print(response.status_code)
soup = BeautifulSoup(response.text, "html.parser")
title = soup.find_all("h1")
subtitle = soup.find_all("h2")
data = {
    "Title": title[0].string,
    "Subtitle": subtitle[0].string,
    "Content": soup.get_text(),
}
print(subtitle[0].string)
# for x in title:
#     print(x)

200
Where to get started with ROS 2


In [ ]:
from clearml import PipelineDecorator

# @PipelineDecorator.component(cache=True,return_values=['data'])
def crawlArticle(link:str):
    import requests
    from bs4 import BeautifulSoup
    response = requests.get(link)
    soup = BeautifulSoup(response.text,"html.parser")
    title = soup.find_all("h1")
    subtitle = soup.find_all("h2")
    data = {
        "URL": link,
        "Title": title[0].string,
        "Subtitle": subtitle[0].string,
        "Content": soup.get_text(),
    }
    return data

# @PipelineDecorator.component(cache=True, return_values=["_id"])
def datatoDB(data:dict):
    import pymongo
    myclient = pymongo.MongoClient("mongodb://localhost:27017/")
    local = myclient["local"]
    col = local["RawData"]
    _id = col.insert_one(data).inserted_id
    return _id

# @PipelineDecorator.component(cache=True, return_values=["doc"])
def confirmDB(_id):
    import pymongo
    myclient = pymongo.MongoClient("mongodb://localhost:27017/")
    local = myclient["local"]
    col = local["RawData"]
    doc = col.find_one({"_id" : _id})
    return doc

# @PipelineDecorator.pipeline(name="ETL Pipeline",project="my project")
def main(link:str):
    data = crawlArticle(link)
    _id = datatoDB(data)
    doc = confirmDB(_id)
    print(doc)

if __name__=="__main__":
    # PipelineDecorator.run_locally()
    main("https://www.ros.org/blog/getting-started/")
    main("https://docs.nav2.org/development_guides/devcontainer_docs/index.html")

    

{'_id': ObjectId('6754dd51af3b5858d805b4f9'), 'URL': 'https://www.ros.org/blog/getting-started/', 'Title': 'Getting Started', 'Subtitle': 'Where to get started with ROS 2', 'Content': '\n\n\n\n\n\n\n\n\n\n\nROS: Getting Started\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nWhy ROS?\nGetting Started\nCommunity\nEcosystem\n\n\n\n\n\n\n\n\n\n\n\nGetting Started\nWhere to get started with ROS 2\n\n\nInstallation\nROS is released as distributions, also called â\x80\x9cdistrosâ\x80\x9d, with more than one ROS distribution supported at a time. Some releases come with long term support (LTS), meaning they are more stable and have undergone extensive testing. Other distributions are newer with shorter lifetimes, but with support for more recent platforms and more recent versions of their constituent ROS packages. See the distributions list for more details. Generally a new ROS distro is released every year on world turtle day, with LTS distros being released in even years. We currently recommend